In [13]:
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

In [28]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, stride, padding, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class ECA(nn.Module):
    def __init__(self, k_size=3):
        super(ECA, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)
        y = self.sigmoid(y)
        return x * y.expand_as(x)

class SGECA(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(SGECA, self).__init__()
        self.pointwise1 = nn.Conv2d(in_channels, in_channels, 1)
        self.dwconv1 = DepthwiseSeparableConv(in_channels, in_channels, stride=stride)
        self.eca = ECA()
        self.dwconv2 = DepthwiseSeparableConv(in_channels, out_channels)
        self.pointwise2 = nn.Conv2d(out_channels, out_channels, 1)  # Corrected to out_channels
        self.stride = stride
        self.shortcut = nn.Sequential()
        if stride == 2 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        # print(f"SGECA input: {x.shape}")
        out = F.relu6(self.pointwise1(x))
        # print(f"SGECA pointwise1: {out.shape}")
        out = self.dwconv1(out)
        # print(f"SGECA dwconv1: {out.shape}")
        out = self.eca(out)
        # print(f"SGECA eca: {out.shape}")
        out = self.dwconv2(out)
        # print(f"SGECA dwconv2: {out.shape}")
        out = F.relu6(self.pointwise2(out))  # Corrected to match out_channels
        # print(f"SGECA pointwise2: {out.shape}")
        out += self.shortcut(x)
        # print(f"SGECA output: {out.shape}")
        return out

class LSGNet(nn.Module):
    def __init__(self, num_classes=8):
        super(LSGNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 24, kernel_size=3, stride=2, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.sg1 = SGECA(24, 116)
        self.sg2 = SGECA(116, 116)
        self.sg3 = SGECA(116, 232, stride=2)
        self.sg4 = SGECA(232, 232)
        self.parcsg1 = SGECA(232, 232)
        self.parcsg2 = SGECA(232, 232)
        self.sg5 = SGECA(232, 464, stride=2)
        self.parcsg3 = SGECA(464, 464)
        self.conv2 = nn.Conv2d(464, 1024, kernel_size=1)
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        # print(f"Input: {x.shape}")
        x = F.relu(self.conv1(x))
        # print(f"After conv1: {x.shape}")
        x = self.maxpool(x)
        # print(f"After maxpool: {x.shape}")
        x = self.sg1(x)
        # print(f"After sg1: {x.shape}")
        x = self.sg2(x)
        # print(f"After sg2: {x.shape}")
        x = self.sg3(x)
        # print(f"After sg3: {x.shape}")
        x = self.sg4(x)
        # print(f"After sg4: {x.shape}")
        x = self.parcsg1(x)
        # print(f"After parcsg1: {x.shape}")
        x = self.parcsg2(x)
        # print(f"After parcsg2: {x.shape}")
        x = self.sg5(x)
        # print(f"After sg5: {x.shape}")
        x = self.parcsg3(x)
        # print(f"After parcsg3: {x.shape}")
        x = F.relu(self.conv2(x))
        # print(f"After conv2: {x.shape}")
        x = self.global_pool(x)
        # print(f"After global_pool: {x.shape}")
        x = torch.flatten(x, 1)
        # print(f"After flatten: {x.shape}")
        x = self.fc(x)
        # print(f"Output: {x.shape}")
        return x


def train(model, train_loader, criterion, optimizer, device, num_epochs):
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    print('Training Finished')

def test(model, test_loader, criterion, device):
    model.to(device)
    model.eval()

    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy

def save_model(model, filename="model_state_dict.pth"):
  torch.save(model, filename)
  torch.save(model.state_dict(), f"s_{filename}.pth")
  
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Define path to PlantVillage dataset
dataset_path_training = './dataset-tomatoes/train'

# Load the PlantVillage dataset with appropriate transforms
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.5),
    transforms.ColorJitter(contrast=0.5),
    transforms.ColorJitter(saturation=0.5),
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.33)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data_transforms_validation_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model = LSGNet(num_classes=8)  # Initialize model
criterion = nn.CrossEntropyLoss()  # Define loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Define optimizer

num_epochs = 150

train_dataset = ImageFolder(root=dataset_path_training, transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Train the model
train(model, train_loader, criterion, optimizer, device, num_epochs)

file_name = f"lsgnet-{num_epochs}-epochs"
save_model(model, file_name)


100%|██████████| 226/226 [03:06<00:00,  1.21it/s]


Epoch 1/150, Loss: 1.3119, Accuracy: 56.76%


100%|██████████| 226/226 [03:03<00:00,  1.23it/s]


Epoch 2/150, Loss: 0.7969, Accuracy: 73.50%


100%|██████████| 226/226 [03:03<00:00,  1.23it/s]


...

Epoch 148/150, Loss: 0.2098, Accuracy: 92.79%


100%|██████████| 226/226 [03:02<00:00,  1.24it/s]


Epoch 149/150, Loss: 0.1989, Accuracy: 93.29%


100%|██████████| 226/226 [03:02<00:00,  1.24it/s]


Epoch 150/150, Loss: 0.2000, Accuracy: 93.15%
Training Finished


In [32]:
validation_dataset = ImageFolder(root='./dataset-tomatoes/validation', transform=data_transforms_validation_test)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=True)
device = "mps" if torch.backends.mps.is_available() else "cpu"

try:
    test(model, validation_loader, criterion, device)
except RuntimeError as e:
    print(f"Runtime error: {e}")
    
# Test Loss: 0.1552, Test Accuracy: 94.38%

100%|██████████| 29/29 [00:09<00:00,  3.08it/s]

Test Loss: 0.1552, Test Accuracy: 94.38%
